In [1]:
import pandas as pd
import numpy as np
import spotipy
import spotipy.util as util
import os 
from spotipy.oauth2 import SpotifyClientCredentials
pd.set_option('display.max_rows', None)
from IPython.display import Audio
import librosa
import sklearn

# Creating the Dataset

#### Spotify API Authentication

In [2]:
client_id = '68d118e35cb84f6089cf7da8b04bcd9a'
client_secret = '2c06256a12bd4ee18f96ff7f44d0177d'
credentials = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
token = credentials.get_access_token()
spotify = spotipy.Spotify(client_credentials_manager=credentials, auth=token)

C:\Users\prave\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  after removing the cwd from sys.path.


#### Get most popular 2 hip-hop songs per year using search

In [3]:
songs = []
for year in range(1980, 2021):
    q_string = 'genre:hip-hop year:{input}'.format(input = year)
    search = spotify.search(q = q_string, type = 'track', limit = 2)
    songs.append(search['tracks']['items'][0])
    songs.append(search['tracks']['items'][1])

#### Take JSON and format into a table

In [ ]:
df = pd.DataFrame()
for song in songs:
    year = song['album']['release_date'][0:4]
    title = song['name']
    artist = song['artists'][0]['name']
    album_name = song['album']['name']
    song_id = song['id']
    df = df.append({'title':title, 'artist':artist, 'album':album_name, 'id':song_id, 'year':year}, ignore_index = True)

#### Replace duplicates and songs that aren't hip hop with 3rd place in each search.

In [ ]:
#0, 3, 6, 7, 10, 32, 38, 64, 65, 72
# df[0] = {'album':'The Sugarhill Gang', 'artist':'The Sugarhill Gang', 'id':'1FnxNtKli7rcB9IU4J8MEg', 'title':'Rapper\'s Delight', 'year':'1980'}
def replace_row(row, album, artist, id, title, year):
    ser = df.loc[row]
    ser['album'] = album
    ser['artist'] = artist
    ser['id'] = id
    ser['title'] = title
    ser['year'] = year
    
replace_row(0, 'The Sugarhill Gang', 'The Sugarhill Gang', '1FnxNtKli7rcB9IU4J8MEg', 'Rapper\'s Delight', 1980)
replace_row(3, '8th Wonder', 'The Sugarhill Gang', '6m0wOLtrzaNwMofVVJTJqs', '8th Wonder', 1981)
replace_row(6, 'Money Dollar Bill Y\'all', 'Jimmy Spicer', '3Bu69QHyCAyBy8TQ8N1lIA', 'Money Dollar Bill Y\'all', 1983)
replace_row(7, 'Sucker D.J.\'s (I Will Survive)', 'Dimples D.', '3GdWz9M6RrwipY72BIdhMA', 'Sucker D.J.\'s (I Will Survive)', 1983)
replace_row(10, 'King of Rock', 'Run-D.M.C.', '7112WRQXlBGe4Os43yw8gV', 'King of Rock', 1985)
replace_row(32, 'It Was Written', 'Nas', '5PQmSHzWnlgG4EBuIqjac2', 'If I Ruled the World (Imagine That)', 1996)
replace_row(38, 'Things Fall Apart', 'The Roots', '4pdLZsxq0y5oJDb6Cxlokw', 'You Got Me', 1999)
replace_row(52, 'Blow the Whistle', 'Too $hort', '2lMg3lCMOGistaWBNGjuT3', 'Blow the Whistle', 2006)
replace_row(64, 'Dreams and Nightmares', 'Meek Mill', '42zd6DYQ4o4SECmTITrM1U', 'Dreams and Nightmares', 2012)
replace_row(65, 'good kid, m.A.A.d city', 'Kendrick Lamar', '2HbKqm4o0w5wEeEFXm2sD4', 'Money Trees', 2012)
replace_row(72, 'EVOL', 'Low Life', '7EiZI6JVHllARrX9PUvAdX', 'Low Life', 2016)

#### And now, add in some songs that either cover influential artists that were originally omitted (2Pac, JAY-Z, Kanye West), as well as a few more interesting or influential songs we'd like to look at.

In [10]:
df = df.append({'album':'My Beautiful Dark Twisted Fantasy', 'artist':'Kanye West', 'id':'22L7bfCiAkJo5xGSQgmiIO', 'title':'All of the Lights', 'year':2010}, ignore_index = True)
df = df.append({'album':'All Eyez On Me', 'artist':'2Pac', 'id':'20fBuVybkHgjF6vNhSMROD', 'title':'Ambitionz Az a Ridah', 'year':1996}, ignore_index = True)
df = df.append({'album':'Greatest Hits', 'artist':'2Pac', 'id':'3ia3dJETSOllPsv3LJkE35', 'title':'California Love', 'year':1998}, ignore_index = True)
df = df.append({'album':'Reasonable Doubt', 'artist':'JAY-Z', 'id':'3nDYsXggRQxf7PCNUjR4rz', 'title':'Dead Presidents II', 'year':1996}, ignore_index = True)
df = df.append({'album':'The Blueprint 3', 'artist':'JAY-Z', 'id':'2igwFfvr1OAGX9SKDCPBwO', 'title':'Empire State of Mind', 'year':2009}, ignore_index = True)
df = df.append({'album':'Fear of a Black Planet', 'artist':'Public Enemy', 'id':'1yo16b3u0lptm6Cs7lx4AD', 'title':'Fight the Power', 'year':1990}, ignore_index = True)
df = df.append({'album':'Late Registration', 'artist':'Kanye West', 'id':'1PS1QMdUqOal0ai3Gt7sDQ', 'title':'Gold Digger', 'year':2005}, ignore_index = True)
df = df.append({'album':'Lil Pump', 'artist':'Lil Pump', 'id':'43ZyHQITOjhciSUUNPVRHc', 'title':'Gucci Gang', 'year':2017}, ignore_index = True)
df = df.append({'album':'DAMN.', 'artist':'Kendrick Lamar', 'id':'7KXjTSCq5nL1LoYtL7XAwS', 'title':'HUMBLE.', 'year':2017}, ignore_index = True)
df = df.append({'album':'Watch the Throne', 'artist':'JAY-Z', 'id':'4Li2WHPkuyCdtmokzW2007', 'title':'Ni**as in Paris', 'year':2011}, ignore_index = True)
df = df.append({'album':'The Chronic', 'artist':'Dr. Dre', 'id':'4YtoipFgf4k0AfD17ZfD5X', 'title':'Nuthin\' But a G Thang', 'year':1992}, ignore_index = True)
df = df.append({'album':'7 EP', 'artist':'Lil Nas X', 'id':'2YpeDb67231RjR0MgVLzsG', 'title':'Old Town Road', 'year':2019}, ignore_index = True)
df = df.append({'album':'ASTROWORLD', 'artist':'Travis Scott', 'id':'2xLMifQCjDGFmkHkpNLD9h', 'title':'SICKO MODE', 'year':2018}, ignore_index = True)
df = df.append({'album':'Mecca and the Soul Brother', 'artist':'Pete Rock', 'id':'2Mb3zpobD0CvJGWv6NpsPy', 'title':'They Reminisce Over You (T.R.O.Y)', 'year':1992}, ignore_index = True)
df.tail()

,title,artist,album,id,year
91,Ni**as in Paris,JAY-Z,Watch the Throne,4Li2WHPkuyCdtmokzW2007,2011
92,Nuthin' But a G Thang,Dr. Dre,The Chronic,4YtoipFgf4k0AfD17ZfD5X,1992
93,Old Town Road,Lil Nas X,7 EP,2YpeDb67231RjR0MgVLzsG,2019
94,SICKO MODE,Travis Scott,ASTROWORLD,2xLMifQCjDGFmkHkpNLD9h,2018
95,They Reminisce Over You (T.R.O.Y),Pete Rock,Mecca and the Soul Brother,2Mb3zpobD0CvJGWv6NpsPy,1992


In [11]:
df

,title,artist,album,id,year
0,Rapper's Delight,The Sugarhill Gang,The Sugarhill Gang,1FnxNtKli7rcB9IU4J8MEg,1980
1,The Breaks,Kurtis Blow,Kurtis Blow,5TvcggVu3s2P1fMAS8BupX,1980
2,Apache,The Sugarhill Gang,8th Wonder,4UCkX8nrBlpxjrrEqtb46a,1981
3,8th Wonder,The Sugarhill Gang,8th Wonder,6m0wOLtrzaNwMofVVJTJqs,1981
4,Planet Rock,Afrika Bambaataa,Planet Rock,3uy0jtkM8QYVTsBazkli1x,1982
5,The Message (feat. Melle Mel & Duke Bootee),Grandmaster Flash & The Furious Five,The Message,5DuTNKFEjJIySAyJH1yNDU,1982
6,Money Dollar Bill Y'all,Jimmy Spicer,Money Dollar Bill Y'all,3Bu69QHyCAyBy8TQ8N1lIA,1983
7,Sucker D.J.'s (I Will Survive),Dimples D.,Sucker D.J.'s (I Will Survive),3GdWz9M6RrwipY72BIdhMA,1983
8,Jam On It,Newcleus,Jam On Revenge,1lB2kyB5h9ceZ388GBfC9L,1984
9,It's Like That,Run–D.M.C.,RUN-DMC (Expanded Edition),2J6QnTjHIWwXErNWyF0RUC,1984


#### Use Spotify's API to pull Spotify's built-in audio features to augument the ones we'll make later.

In [12]:
spotify_feature_df = pd.DataFrame()
for fl in spotify.audio_features(tracks = list(df['id'])):
    spotify_feature_df = spotify_feature_df.append({'danceability':fl['danceability'],
                              'energy':fl['energy'],
                              'loudness':fl['loudness'],
                              'speechiness':fl['speechiness'],
                              'acousticness':fl['acousticness'],
                             'valence':fl['valence'],
                             'tempo':fl['tempo']}, ignore_index = True)

In [13]:
spotify_feature_df.head()

,acousticness,danceability,energy,loudness,speechiness,tempo,valence
0,0.2200,0.905,0.692,-14.143,0.199,111.292,0.943
1,0.0987,0.910,0.576,-13.256,0.109,112.877,0.907
2,0.3320,0.827,0.928,-7.266,0.133,115.023,0.261
3,0.2980,0.871,0.937,-8.536,0.195,106.494,0.675
4,0.0110,0.915,0.807,-9.636,0.130,127.350,0.708


#### To match these songs with the audio files, we're going to have to find matches in strings, and manually cover the rest

In [14]:
matches = [0] * len(df)
for i in range(len(df)):
    song = df['title'][i]
    for name in os.listdir('./data'):
        if name.lower()[:-4] in song.lower():
            matches[i] = './data/' + name

In [15]:
df['file'] = pd.Series(matches)
df

,title,artist,album,id,year,file
0,Rapper's Delight,The Sugarhill Gang,The Sugarhill Gang,1FnxNtKli7rcB9IU4J8MEg,1980,./data/Rapper's Delight.mp3
1,The Breaks,Kurtis Blow,Kurtis Blow,5TvcggVu3s2P1fMAS8BupX,1980,./data/The Breaks.mp3
2,Apache,The Sugarhill Gang,8th Wonder,4UCkX8nrBlpxjrrEqtb46a,1981,./data/Apache.mp3
3,8th Wonder,The Sugarhill Gang,8th Wonder,6m0wOLtrzaNwMofVVJTJqs,1981,./data/8th Wonder.mp3
4,Planet Rock,Afrika Bambaataa,Planet Rock,3uy0jtkM8QYVTsBazkli1x,1982,./data/Planet Rock.mp3
5,The Message (feat. Melle Mel & Duke Bootee),Grandmaster Flash & The Furious Five,The Message,5DuTNKFEjJIySAyJH1yNDU,1982,./data/The Message.mp3
6,Money Dollar Bill Y'all,Jimmy Spicer,Money Dollar Bill Y'all,3Bu69QHyCAyBy8TQ8N1lIA,1983,./data/Money Dollar Bill Y'All.mp3
7,Sucker D.J.'s (I Will Survive),Dimples D.,Sucker D.J.'s (I Will Survive),3GdWz9M6RrwipY72BIdhMA,1983,./data/Sucker D.J.'s (I Will Survive).mp3
8,Jam On It,Newcleus,Jam On Revenge,1lB2kyB5h9ceZ388GBfC9L,1984,./data/Jam On It.mp3
9,It's Like That,Run–D.M.C.,RUN-DMC (Expanded Edition),2J6QnTjHIWwXErNWyF0RUC,1984,./data/It's Like That.mp3


#### Phew. That took a while. Now onto the actual feature extraction.

In [31]:
ss = sklearn.preprocessing.StandardScaler()
def filename_to_features(fname):
    song, rate = librosa.load(fname)
    print(fname)

    # 12 MFCC's
    mfcc_series = pd.DataFrame(librosa.feature.mfcc(y = song, sr = rate, n_mfcc = 12).T).mean()

    # 12 chroma features
    chroma_series = pd.DataFrame(librosa.feature.chroma_stft(y = song, sr = rate)).mean(axis = 1)

    zcr = pd.Series(np.mean(librosa.feature.zero_crossing_rate(y = song)))
    bandwidth = pd.Series(np.mean(librosa.feature.spectral_bandwidth(y = song)))
    centroid = pd.Series(np.mean(librosa.feature.spectral_centroid(y = song)))
    rolloff = pd.Series(np.mean(librosa.feature.spectral_rolloff(y = song)))
    return pd.concat([mfcc_series, chroma_series, zcr, bandwidth, centroid, rolloff]).reset_index(drop = True)

In [ ]:
features = df['file'].apply(filename_to_features)

Standardize values:

In [43]:
features = pd.DataFrame(ss.fit_transform(features))

In [46]:
features.to_csv('audio_features.csv')

In [47]:
full_features = pd.concat([df, features, spotify_feature_df], axis = 1)
full_features.to_csv('full_data.csv')

In [49]:
pd.read_csv('full_data.csv')

,Unnamed: 0,title,artist,album,id,year,file,mfcc_0,mfcc_1,mfcc_2,...,bandwidth,centroid,rolloff,acousticness,danceability,energy,loudness,speechiness,tempo,valence
0,0,Rapper's Delight,The Sugarhill Gang,The Sugarhill Gang,1FnxNtKli7rcB9IU4J8MEg,1980,./data/Rapper's Delight.mp3,-1.520227,-1.262824,1.255192,...,1.840998,1.657349,1.834801,0.220000,0.905,0.692,-14.143,0.1990,111.292,0.9430
1,1,The Breaks,Kurtis Blow,Kurtis Blow,5TvcggVu3s2P1fMAS8BupX,1980,./data/The Breaks.mp3,-1.019009,0.545576,-0.092856,...,0.342726,-0.268865,0.025154,0.098700,0.910,0.576,-13.256,0.1090,112.877,0.9070
2,2,Apache,The Sugarhill Gang,8th Wonder,4UCkX8nrBlpxjrrEqtb46a,1981,./data/Apache.mp3,0.704940,-0.689719,-1.794352,...,0.180036,0.880376,0.655159,0.332000,0.827,0.928,-7.266,0.1330,115.023,0.2610
3,3,8th Wonder,The Sugarhill Gang,8th Wonder,6m0wOLtrzaNwMofVVJTJqs,1981,./data/8th Wonder.mp3,0.443494,-1.131487,-2.393582,...,0.186680,1.368371,0.869361,0.298000,0.871,0.937,-8.536,0.1950,106.494,0.6750
4,4,Planet Rock,Afrika Bambaataa,Planet Rock,3uy0jtkM8QYVTsBazkli1x,1982,./data/Planet Rock.mp3,-0.356059,-2.095488,-0.310716,...,1.299604,2.193667,1.856558,0.011000,0.915,0.807,-9.636,0.1300,127.350,0.7080
5,5,The Message (feat. Melle Mel & Duke Bootee),Grandmaster Flash & The Furious Five,The Message,5DuTNKFEjJIySAyJH1yNDU,1982,./data/The Message.mp3,-0.851803,-1.093314,-0.571710,...,0.341781,0.937769,0.697060,0.024900,0.947,0.607,-10.580,0.2020,100.619,0.7320
6,6,Money Dollar Bill Y'all,Jimmy Spicer,Money Dollar Bill Y'all,3Bu69QHyCAyBy8TQ8N1lIA,1983,./data/Money Dollar Bill Y'All.mp3,-0.151258,1.602239,0.676673,...,-1.395945,-1.598583,-1.728273,0.030300,0.862,0.669,-10.260,0.1580,99.379,0.6680
7,7,Sucker D.J.'s (I Will Survive),Dimples D.,Sucker D.J.'s (I Will Survive),3GdWz9M6RrwipY72BIdhMA,1983,./data/Sucker D.J.'s (I Will Survive).mp3,-1.232624,-0.421354,-1.541503,...,-1.087991,-0.268505,-0.635795,0.038000,0.868,0.832,-9.080,0.1360,107.979,0.9310
8,8,Jam On It,Newcleus,Jam On Revenge,1lB2kyB5h9ceZ388GBfC9L,1984,./data/Jam On It.mp3,0.408623,-0.670100,1.178526,...,1.735333,1.124228,1.510166,0.003020,0.804,0.782,-6.224,0.0606,116.806,0.3460
9,9,It's Like That,Run–D.M.C.,RUN-DMC (Expanded Edition),2J6QnTjHIWwXErNWyF0RUC,1984,./data/It's Like That.mp3,-0.637880,-1.068307,-1.731048,...,-0.059085,0.942822,0.640400,0.059100,0.964,0.889,-5.169,0.1470,121.144,0.5520
